In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [2]:
data = pd.read_csv('Road_Distance.csv')

In [4]:
data.head()

,Distance in Kilometres,Ahmedabad,Bangalore,Bhubaneshwar,Bombay,Calcutta,Chandigarh,Cochin,Delhi,Hyderabad,...,Jaipur,Kanpur,Lucknow,Madras,Nagpur,Nasik,Panjim,Patna,Pondicherry,Pune
0,Agartala,3305,3824,2286,3593,1863,2998,4304,2708,3330,...,2801,2281,2252,3493,2696,3365,3507,1681,3661,3442
1,Agra,878,1848,1578,1202,1300,448,2278,200,1246,...,230,290,369,2048,770,1005,1715,885,2210,1214
2,Ahmedabad,-,1490,1697,552,2068,1157,1845,911,1436,...,648,1168,1247,1821,965,504,1165,1656,1818,664
3,Allahabad,1251,1686,1090,1457,817,912,2216,650,1084,...,713,193,234,2011,608,1155,1419,402,1077,1364
4,Amritsar,1356,2496,2224,1849,1919,239,3163,445,1892,...,706,926,939,2688,1416,1665,2237,1531,2856,1862


In [26]:
label_to_city = data.iloc[:, 0].to_dict()
print(label_to_city)

city_to_label = {v: k for k, v in label_to_city.items()}
print(city_to_label)

{0: 'Agartala', 1: 'Agra', 2: 'Ahmedabad', 3: 'Allahabad', 4: 'Amritsar', 5: 'Asansol', 6: 'Bangalore', 7: 'Baroda', 8: 'Bhopal', 9: 'Bhubaneshwar', 10: 'Bombay', 11: 'Calcutta', 12: 'Calicut', 13: 'Chandigarh', 14: 'Cochin', 15: 'Coimbatore', 16: 'Delhi', 17: 'Gwalior', 18: 'Hubli', 19: 'Hyderabad', 20: 'Imphal', 21: 'Indore', 22: 'Jabalpur', 23: 'Jaipur', 24: 'Jamshedpur', 25: 'Jullundur', 26: 'Kanpur', 27: 'Kolhapur', 28: 'Lucknow', 29: 'Ludhiana', 30: 'Madras', 31: 'Madurai', 32: 'Meerut', 33: 'Nagpur', 34: 'Nasik', 35: 'Panjim', 36: 'Patna', 37: 'Pondicherry', 38: 'Pune', 39: 'Ranchi', 40: 'Shillong', 41: 'Shimla', 42: 'Surat', 43: 'Trivandrum', 44: 'Varanasi', 45: 'Vijayawada', 46: 'Vishakapatnam'}
{'Agartala': 0, 'Agra': 1, 'Ahmedabad': 2, 'Allahabad': 3, 'Amritsar': 4, 'Asansol': 5, 'Bangalore': 6, 'Baroda': 7, 'Bhopal': 8, 'Bhubaneshwar': 9, 'Bombay': 10, 'Calcutta': 11, 'Calicut': 12, 'Chandigarh': 13, 'Cochin': 14, 'Coimbatore': 15, 'Delhi': 16, 'Gwalior': 17, 'Hubli': 18, '

In [10]:
graph = np.zeros((len(city_to_label), len(city_to_label)))
graph.shape

(47, 47)

In [17]:
for index, row in data.iterrows():
    for column, value in row.items():
        
        if column != 'Distance in Kilometres' and value != '-':
            graph[index, city_to_label[column]] = int(value)


array([[   0.,    0., 3305., ...,    0.,    0.,    0.],
       [   0.,    0.,  878., ...,    0.,    0.,    0.],
       [   0.,    0.,    0., ...,    0.,    0.,    0.],
       ...,
       [   0.,    0., 1373., ...,    0.,    0.,    0.],
       [   0.,    0., 1705., ...,    0.,    0.,    0.],
       [   0.,    0., 1815., ...,    0.,    0.,    0.]])

In [21]:
graph[5][5]

0.0

### UNIFROM COST SEARCH

In [27]:
import heapq

def uniormCostSearch(graph, source, dest):
    n = len(graph)
    dist = [float('inf')] * n
    prev = [None] * n
    visited = [False] * n
    dist[source] = 0
    heap = [(0, source)]
    
    while heap:
        (d, u) = heapq.heappop(heap)
        if visited[u]:
            continue
        visited[u] = True
        
        if u == dest:
            path = []
            while prev[u] is not None:
                path.append(label_to_city[u])
                u = prev[u]

            path.append(label_to_city[source])
            path.reverse()
            return dist[dest], path
        
        for v in range(n):
            if graph[u][v] != 0:
                alt = dist[u] + graph[u][v]
                if alt < dist[v]:
                    dist[v] = alt
                    prev[v] = u
                    heapq.heappush(heap, (dist[v], v))
    
    return float('inf'), ['No path found']


In [28]:
uniormCostSearch(graph, city_to_label['Bombay'], city_to_label['Delhi'])

(1257.0, ['Bombay', 'Indore', 'Jaipur', 'Delhi'])

### A* With admissible(Straigh line distance heuristic)

In [29]:
heuristic = np.zeros((len(city_to_label), len(city_to_label)))

In [30]:
dheuristic = pd.read_csv('heuristic.csv')
dheuristic.head()

,agartala,agartala.1,0
0,agartala,agra,2516
1,agartala,ahmedabad,3255
2,agartala,allahabad,2033
3,agartala,amritsar,3101
4,agartala,asansol,2026


In [33]:
for index, row in dheuristic.iterrows():
    row_tuple = tuple(row)
    heuristic[city_to_label[row_tuple[0].capitalize()], city_to_label[row_tuple[1].capitalize()]] = row_tuple[2]

heuristic

array([[   0., 2516., 3255., ..., 1911., 3011., 2681.],
       [2516.,    0.,  828., ...,  555., 1462., 1630.],
       [3255.,  828.,    0., ..., 1317., 1428., 1747.],
       ...,
       [1911.,  555., 1317., ...,    0., 1405., 1363.],
       [3011., 1462., 1428., ..., 1405.,    0.,  818.],
       [2681., 1630., 1747., ..., 1363.,  818.,    0.]])

In [34]:
def astar(graph, heuristic, source, dest):
    n = len(graph)
    dist = [float('inf')] * n
    prev = [None] * n
    visited = [False] * n
    dist[source] = 0
    heap = [(0, source)]
    
    while heap:
        (d, u) = heapq.heappop(heap)
        if visited[u]:
            continue
        visited[u] = True
        
        if u == dest:
            path = []
            while prev[u] is not None:
                path.append(label_to_city[u])
                u = prev[u]

            path.append(label_to_city[source])
            path.reverse()
            return dist[dest], path
        
        for v in range(n):
            if graph[u][v] != 0:
                alt = dist[u] + graph[u][v]
                if alt < dist[v]:
                    dist[v] = alt
                    prev[v] = u
                    heapq.heappush(heap, (dist[v] + heuristic[v][dest], v))
    
    return float('inf'), ['No path found']


In [36]:
astar(graph,heuristic ,city_to_label['Bombay'], city_to_label['Delhi'])

(1257.0, ['Bombay', 'Indore', 'Jaipur', 'Delhi'])

### A* with inadmissible search

In [37]:
import random

def astar2(graph, heuristic, source, dest):
    n = len(graph)
    dist = [float('inf')] * n
    prev = [None] * n
    visited = [False] * n
    dist[source] = 0
    heap = [(0, source)]
    
    while heap:
        (d, u) = heapq.heappop(heap)
        if visited[u]:
            continue
        visited[u] = True
        
        if u == dest:
            path = []
            while prev[u] is not None:
                path.append(label_to_city[u])
                u = prev[u]

            path.append(label_to_city[source])
            path.reverse()
            return dist[dest], path
        
        for v in range(n):
            if graph[u][v] != 0:
                alt = dist[u] + graph[u][v]
                if alt < dist[v]:
                    dist[v] = alt
                    prev[v] = u
                    interm = random.randint(0, 46)
                    heapq.heappush(heap, (dist[v] + heuristic[v][interm] + heuristic[interm][dest], v))

    return float('inf'), ['No path found']


In [38]:
astar2(graph,heuristic ,city_to_label['Bombay'], city_to_label['Delhi'])

(1395.0, ['Bombay', 'Indore', 'Delhi'])